<a href="https://colab.research.google.com/github/OstrovskiyMaxim/.NETCourse/blob/master/lite/stable_diffusion_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Change working directory
%cd /content

# Suppress TF log messages
%env TF_CPP_MIN_LOG_LEVEL=1

# Update system packages
!apt -y update -qq

# Install Python 3.10 and dev tools
!apt -y install python3.10 python3.10-dev python3.10-distutils pkg-config libcairo2-dev aria2 -qq

# Switch python3 to 3.10
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 2
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!update-alternatives --config python3 <<< "1"
!python3 --version

# Install pip for Python 3.10
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py

# Install PyTorch + related packages
!python3 -m pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U

# Install other Python packages
!python3 -m pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U

# Download gperftools for memory optimization
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

# Clone WebUI and embeddings
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive

# Download ESRGAN model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth

# Download helper scripts
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py

# Clone extensions
EXTENSIONS=(
    "deforum-for-automatic1111-webui"
    "stable-diffusion-webui-images-browser"
    "stable-diffusion-webui-huggingface"
    "sd-civitai-browser"
    "sd-webui-additional-networks"
    "sd-webui-tunnels"
    "batchlinks-webui"
    "stable-diffusion-webui-catppuccin"
    "stable-diffusion-webui-rembg"
    "stable-diffusion-webui-two-shot"
    "sd-webui-aspect-ratio-helper"
    "asymmetric-tiling-sd-webui"
)

for ext in "${EXTENSIONS[@]}"; do
    !git clone -b v2.4 https://github.com/camenduru/$ext /content/stable-diffusion-webui/extensions/$ext
done

# Move into WebUI directory
%cd /content/stable-diffusion-webui

# Reset Git (safety)
!git reset --hard

# Download Stable Diffusion checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd14/resolve/main/sd-v1-4.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o sd-v1-4.ckpt

# Apply some WebUI fixes
!sed -i -e '''/from modules import launch_utils/a\import os''' /content/stable-diffusion-webui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared.py

# Launch WebUI
!python3 launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple --skip-python-version-check
